# Scripts Antoine

In [2]:
# Parameters
file_location = r"C:\fichier.jpg"
# file_location = "images/20 mins_t9.jpg"
particle_pixel_size = 8

## Data Processing

In [5]:
import numpy as np
import trackpy as tp
import pandas as pd
from PIL import Image
from skimage.exposure import equalize_adapthist
import functions as fun

# Opening, preprocessing the image
img = Image.open(file_location, mode='r')
img_array = np.array(img)
img_norm = equalize_adapthist(img_array)
img_info = {'height' : np.shape(img_array)[0], 'width' : np.shape(img_norm)[1]}

# Particle detection
locations_comp = tp.locate(img_norm, 9, minmass=4, separation=9, invert=True)
locations = tp.locate(img_norm, 9, minmass=1, separation=5, invert=True, threshold=0.05)

# Particle pair computation
rho_ctrs, g = fun.particle_pair(locations, img_info, particle_pixel_size, rhos=np.linspace(0.01,5*particle_pixel_size, 100))

def export_g_rho(rho_ctrs, g, file_name='data.csv'):
    data = pd.DataFrame({'rho':rho_ctrs, 'g':g})
    data.to_csv(file_name)

export_g_rho(rho_ctrs, g)
print(' > {:05d} particles found'.format(len(locations)))

 > 02149 particles found


## Plotting figure with particle positions

In [4]:
from bokeh.plotting import figure, show

f = figure(width=1000, height=1000, tooltips=[('x', '$x'),('y', '$y'),('lum', '@image')])
f.image([1-img_norm], x=0, y=0, dw=img_info['width'], dh=img_info['height'])
f.scatter(locations_comp['x'], locations_comp['y'], marker='+', line_color='lime', size=5)
f.scatter(locations['x'], locations['y'], marker='x', line_color='red', size=5)
show(f)

## Plotting figure with $g(\rho)$

In [24]:
from bokeh.plotting import figure, show

f = figure()
f.line(rho_ctrs, g)
f.line([1,1], [0,np.max(g)], line_dash='dotted', line_color='black')
f.line(rho_ctrs, np.ones(np.size(rho_ctrs)), line_dash='dashed', line_color='black')
show(f)

## 'Mass' of particles

In [19]:
from bokeh.plotting import figure, show

mss, bins = np.histogram(locations['mass'], bins=100)
f = figure()
f.quad(left=bins[:-1], right=bins[1:], top=mss, bottom=0*mss)
show(f)